In [1]:
path = "/Users\Brandon\Desktop\Classes\E208\Data\swg_camera_traps.json"
newpath = "/Users\Brandon\Desktop\Classes\E208\Data\.json"

annotations =  "/Users\Brandon\Desktop\Classes\E208\Data/annotations.json"
annotations_mongoose = "/Users\Brandon\Desktop\Classes\E208\Data/annotations_crab_eating_mongoose.json"
images = annotations =  "/Users\Brandon\Desktop\Classes\E208\Data/images.json"
images_mongoose = "/Users\Brandon\Desktop\Classes\E208\Data/images_crab_eating_mongoose.json"

mongoose_download = "/Users\Brandon\Desktop\Classes\E208\Data/crab_eating_mongoose/"

In [12]:
import json
import os
import subprocess
from tqdm import tqdm

# Path to your JSON file
json_file_path = images_mongoose
# Base URL for the images
base_url = 'gs://public-datasets-lila/swg-camera-traps/'  # Update this URL as needed
# Local directory to save images
download_directory = mongoose_download

# Ensure the download directory exists
os.makedirs(download_directory, exist_ok=True)

# Read the JSON file
with open(json_file_path, 'r') as file:
    images = json.load(file)



seq_ids = {}

image_url_list = []
# Iterate over the images and download each one
for image in tqdm(images):
    seq_id = image['seq_id']
    num_frames = image['seq_num_frames']
    frame_num = image['frame_num']
    file_name = image['file_name']
    if seq_id not in seq_ids:
        seq_ids[seq_id] = [False, num_frames]

    
    if not seq_ids[seq_id][0]: #onyl download one sequence per image, and pass if we already downloaded the image
        if frame_num == seq_ids[seq_id][1]//2: #Only download the image in the middle of the sequence
            seq_ids[seq_id][0] = True #We've downloaded the image
            image_url_list.append(file_name)

#file_name = image['file_name']
#image_url = base_url + file_name
#local_file_path = os.path.join(download_directory, file_name.split('/')[-1])

#Construct the AzCopy command
#command = f'gsutil -m cp -r "{image_url}" "{local_file_path}"'

#Execute the command
#subprocess.run(command, shell=True)


100%|████████████████████████████████████████████████████████████████████████| 17305/17305 [00:00<00:00, 435070.83it/s]


In [13]:
#now we download all our images
for file_name in tqdm(image_url_list):
    image_url = base_url + file_name
    local_file_path = os.path.join(download_directory, file_name.split('/')[-1])
    
    
    command = f'gsutil -m cp -r "{image_url}" "{local_file_path}"'
    

    subprocess.run(command, shell=True)



  0%|▎                                                                             | 21/5046 [01:45<6:59:42,  5.01s/it]


KeyboardInterrupt: 

In [64]:
#This is the code to extract the image data from the filtered annotations data

import json

annotations_file_path = annotations_mongoose
images_file_path = images
output_file_path = newpath

# Step 1: Read annotations_4.json and create a set of image IDs
with open(annotations_file_path, 'r') as file:
    annotations = json.load(file)
    image_ids = set(ann['image_id'] for ann in annotations)

# Step 2: Read images.json and filter out images with IDs in the set
with open(images_file_path, 'r') as file:
    images = json.load(file)
    filtered_images = [img for img in images if img['id'] in image_ids]

# Step 3: Write the filtered images to a new JSON file
with open(output_file_path, 'w') as file:
    json.dump(filtered_images, file, indent=4)




In [59]:
#this is the code to extract all the ID's from the annotations. 
#4 = crab_eating_mongoose
import json

annotations_file_path = annotations
new_json_file_path = newpath

# Read the annotations JSON file
with open(annotations_file_path, 'r') as file:
    annotations = json.load(file)

# Filter annotations with category_id of 4
filtered_annotations = [annotation for annotation in annotations if annotation['category_id'] == 4]

# Write the filtered annotations to a new JSON file
with open(new_json_file_path, 'w') as new_file:
    json.dump(filtered_annotations, new_file, indent=4)





In [54]:
#This is the code to extract one of the original 4 categories to a new .json. 

import ijson

json_file_path = path
new_json_file_path = newpath

# Opening the original JSON file
with open(json_file_path, 'r') as file:
    # Using ijson to parse only the categories section of the JSON file
    categories = ijson.items(file, 'images.item')

    # Converting the generator to a list
    category_list = list(categories)

# Writing the collected categories to a new JSON file
with open(new_json_file_path, 'w') as new_file:
    json.dump(category_list, new_file, indent=4)


In [47]:
import ijson
import json
from tqdm import tqdm

json_file_path = path
new_json_file_path = newpath

# Opening the original JSON file
with open(json_file_path, 'r') as file:
    # Using ijson to parse the JSON file iteratively
    parser = ijson.parse(file)

    categories = []

    # Iterating through the parser
    num = 0
    for prefix, event, value in tqdm(parser):
        num += 1
        if prefix == 'categories.item':
            if event == 'start_map':
                # Start a new category item
                current_category = {}
            elif event == 'map_key':
                # The current key in the category item
                current_key = value
            elif event in ['string', 'number']:
                # Adding key-value pair to the category item
                current_category[current_key] = value
            elif event == 'end_map':
                # End of the current category item, add it to the categories list
                categories.append(current_category)

# Writing the collected categories to a new JSON file
with open(new_json_file_path, 'w') as new_file:
    json.dump(categories, new_file, indent=4)


65270008it [01:17, 839194.60it/s] 


In [53]:
import ijson
import json

json_file_path = path
new_json_file_path = newpath

# Opening the original JSON file
with open(json_file_path, 'r') as file:
    # Using ijson to parse the JSON file iteratively
    parser = ijson.parse(file)

    data = []
    current_item = {}
    count = 0

    for prefix, event, value in parser:
        # Building each item from parts
        if event == 'map_key':
            current_key = value
        elif event in ['string', 'number', 'boolean']:
            current_item[current_key] = value
        elif event == 'end_map':
            data.append(current_item)
            current_item = {}
            count += 1
            if count >= 10:
                break

# Writing the collected data to a new JSON file
with open(new_json_file_path, 'w') as new_file:
    json.dump(data, new_file, indent=4)


In [23]:

import ijson

json_file_path = path

with open(json_file_path, 'r') as file:
    # Using ijson to parse the JSON file iteratively
    parser = ijson.parse(file)

    # Iterating through the parser
    num = 0
    for prefix, event, value in parser:
        print("prefix", prefix, "event", event, "value", value)
        num +=1
        if num > 100:
            break


prefix  event start_map value None
prefix  event map_key value info
prefix info event start_map value None
prefix info event map_key value year
prefix info.year event number value 2021
prefix info event map_key value version
prefix info.version event string value 1.0
prefix info event map_key value description
prefix info.description event string value SWG Camera Traps
prefix info event map_key value contributor
prefix info.contributor event string value Saola Working Group
prefix info event map_key value date_created
prefix info.date_created event string value 2021-03-24
prefix info event end_map value None
prefix  event map_key value images
prefix images event start_array value None
prefix images.item event start_map value None
prefix images.item event map_key value id
prefix images.item.id event string value c7a2830c-8c29-11eb-860b-000d3a74c7de
prefix images.item event map_key value datetime
prefix images.item.datetime event string value 2017-08-15 11:48:11+00:00
prefix images.item 